## Generating Text using LSTM
**Result:**
- **seed**: was just in time to hear it s

- **output**: olethmy **and** bs iave aspu wale tuch lnlely **and see as** oncered werpenceradle **of** lt a *vhry* clarfe of toame **of the** werling **but the** waid **to the** wenl **of the** senembobtooo **as** *homd* **the** afded **a said on her** fach **on** *wher stpp* anice **as** speshngs **the** moew **the** waid **to alice said** bester whe hrownan **and** oore tatsed **by** tuatted poly **as** umeer **the said** tomeiers rres tuleking **to the** tueenened **of** toeased herser tenarked **out** *repembir* ins tie *finis* int ttill ie sayher **when** tuch a pear *hirself* anice **as** snee to deer or sle

**Approach:**
- Added **'#'** character as padding with index value=0
- Used 2 LSTMs with **Masking Layer** to skip '#'(padding) while training
- Used **Dropout everywhere** for input & output vector units
- **No Dropout** before Dense layer 

**Loss: 0.7253**

**Total Epochs: 100, Batch Size:384**

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import sys
import string
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Masking
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.utils import np_utils

Using TensorFlow backend.


#### Read Text file

In [2]:
# load ascii text and covert to lowercase
filename = "mystory.txt"
raw_text = open(filename,encoding='utf-8').read()
raw_text = raw_text.lower()

In [4]:
# Total number of line in text
total_lines = 0
with open(filename, 'r') as f:
    for line in f:
        total_lines += 1
print("Total Lines:",total_lines )

Total Lines: 3328


#### Data Preprocessing
- Remove Punctuation from string
- Remove Blank line from string
- char to int & int to char array for a-z & ' '
- Create Input (dataX) & output label (dataY)
- Add padding if input string length less than 72 (Maxlength of string)
- Reshape Input to [samples, time steps, features]
- Normalize Input X & One hot vector representation of Y

In [5]:
# Punctuation Removal funcation 
def Punctuation_Removal(str_value): 
  
    punctuation_list = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'
  
    for x in str_value.lower(): 
        if x in punctuation_list: 
            str_value = str_value.replace(x, "") 
  
    return str_value

In [6]:
# ~~~~~~~~~~~ Remove Punctuation from string ~~~~~~~~~~
mytext = Punctuation_Removal(raw_text)
mytext = mytext.replace('\n','.')
mytext = mytext.replace('\ufeff','')
mytext = mytext.split('.')
print("Total line with blanks: "+str(len(mytext)))

# ~~~~~~~~~~ Remove Blank line from text ~~~~~~~~~~~
final_text = []
for line in mytext:
  if line == '':
    continue
  final_text.append(line)
print("Final Total Line: "+str(len(final_text)))   

Total line with blanks: 4317
Final Total Line: 2845


In [7]:
# Find unique value into text:  'A-Z' +  ' '

mystr = ' '.join(final_text) #join all sentences
chars = sorted(list(set(mystr))) #Find unique value & sorting them
print("Unique Character list: "+ '\n'+ str(chars) )

Unique Character list: 
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [8]:
# ~~~~~~~ char_to_int ~~~~~~~~~~
char_to_int = dict((c, i) for i, c in enumerate(chars,1)) #start enum with 1 value

# ~~~ Used '#' as a padding letter ~~
char_to_int['#'] = 0
print("Adding \'#\' as padding character \n")
print("Char_to_int array: "+'\n'+ str(char_to_int) + '\n')


# ~~~~~~~~ int_to_char ~~~~~~~~~
int_to_char = {v: k for k, v in char_to_int.items()}
print("int_to_char array: "+ '\n' + str(int_to_char))

Adding '#' as padding character 

Char_to_int array: 
{' ': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27, '#': 0}

int_to_char array: 
{1: ' ', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e', 7: 'f', 8: 'g', 9: 'h', 10: 'i', 11: 'j', 12: 'k', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'p', 18: 'q', 19: 'r', 20: 's', 21: 't', 22: 'u', 23: 'v', 24: 'w', 25: 'x', 26: 'y', 27: 'z', 0: '#'}


In [9]:
# Create Input dataX & output label dataY
dataX = []
dataY = []

for line in final_text:
  for i in range(1,len(line)):
    seq_in = line[:i]
    seq_out = line[i]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
  

In [10]:
num = 4
print("First {} lines of story:".format(num)+'\n')
for i in range(num):
  print(final_text[i])

First 4 lines of story:

chapter i
 down the rabbithole
alice was beginning to get very tired of sitting by her sister on the
bank and of having nothing to do once or twice she had peeped into the


In [11]:
print("Input & Output Pair\n")
for i in range(9,35):
  print("input:", ''.join([int_to_char[j]  for j in list(dataX[i])]) ,
    "    output:",''.join(int_to_char[dataY[i]]))

Input & Output Pair

input:  d     output: o
input:  do     output: w
input:  dow     output: n
input:  down     output:  
input:  down      output: t
input:  down t     output: h
input:  down th     output: e
input:  down the     output:  
input:  down the      output: r
input:  down the r     output: a
input:  down the ra     output: b
input:  down the rab     output: b
input:  down the rabb     output: i
input:  down the rabbi     output: t
input:  down the rabbit     output: h
input:  down the rabbith     output: o
input:  down the rabbitho     output: l
input:  down the rabbithol     output: e
input: a     output: l
input: al     output: i
input: ali     output: c
input: alic     output: e
input: alice     output:  
input: alice      output: w
input: alice w     output: a
input: alice wa     output: s


In [12]:
# ~~~~~~~~ Find Max line length in text ~~~~~~~~
maxlen=0
for line in dataX:
  if maxlen<len(line):
    maxlen=len(line)
print("Max lenght of line: "+str(maxlen))

# ~~~~~~~~ Add Padding for line < maxlen(=72 in our case) ~~~~~~~
dataX_padded = pad_sequences(dataX, maxlen=maxlen, padding='post',value=0,dtype='int32')

Max lenght of line: 72


In [13]:
n_patterns = len(dataX_padded)
print("Total Patterns/Input in our dataset: ", n_patterns)

n_vocab = len(chars) # chars = list of unique values
print("Total vocab ['a-z' and ' '] excluding padding \'#\' :" , n_vocab)

print("\ndataX_padded:\n"+str(dataX_padded))


Total Patterns/Input in our dataset:  129829
Total vocab ['a-z' and ' '] excluding padding '#' : 27

dataX_padded:
[[ 4  0  0 ...  0  0  0]
 [ 4  9  0 ...  0  0  0]
 [ 4  9  2 ...  0  0  0]
 ...
 [19  6 14 ...  0  0  0]
 [19  6 14 ...  0  0  0]
 [19  6 14 ...  0  0  0]]


In [18]:
# reshape X to be [samples, time steps, features]
features=1
X = np.reshape(dataX_padded, (n_patterns, maxlen, features))

# normalize input X
X = X / float(n_vocab)

# one hot encode the output variable
Y = np_utils.to_categorical(dataY)

print("Shape:           (sample,steps,features)")
print("Shape of input X: ", str(X.shape))
print("Shape of output Y: ",str(Y.shape))

Shape:           (sample,steps,features)
Shape of input X:  (129829, 72, 1)
Shape of output Y:  (129829, 28)


### Train Model

- `Masking Layer`: Skip padding char using masking value=0 while training

- Use `2 LSTMs with dropout everywhere` (Input vector & output vector units) 

- `No Dropout` before Dense layer

In [36]:
model = Sequential()

model.add(Masking(mask_value=0, input_shape=(maxlen, features)))

model.add(LSTM(256,  dropout=0.1, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256,  dropout=0.1))

model.add(Dense(Y.shape[1], activation='softmax')) # Y.shape[1] = output char


In [37]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_5 (Masking)          (None, 72, 1)             0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 72, 256)           264192    
_________________________________________________________________
dropout_5 (Dropout)          (None, 72, 256)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_5 (Dense)              (None, 28)                7196      
Total params: 796,700
Trainable params: 796,700
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001))

# define the checkpoint
filepath="weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min', period=25)
callbacks_list = [checkpoint]

print("Learning Rate1= 0.001 ")
print ("Epoch 1 to 50 \n")
# fit the model : 
model.fit(X, Y, epochs=50, batch_size=384, callbacks=callbacks_list)

Learning Rate1= 0.001 
Epoch 1 to 50 

Epoch 1/50
129829/129829 [==============================] - 42s 320us/step - loss: 2.8241
Epoch 2/50
129829/129829 [==============================] - 40s 308us/step - loss: 2.6064
Epoch 3/50
129829/129829 [==============================] - 40s 309us/step - loss: 2.3865
Epoch 4/50
129829/129829 [==============================] - 40s 308us/step - loss: 2.2233
Epoch 5/50
129829/129829 [==============================] - 40s 308us/step - loss: 2.1014
Epoch 6/50
129829/129829 [==============================] - 40s 308us/step - loss: 2.0005
Epoch 7/50
129829/129829 [==============================] - 40s 308us/step - loss: 1.9208
Epoch 8/50
129829/129829 [==============================] - 40s 308us/step - loss: 1.8553
Epoch 9/50
129829/129829 [==============================] - 40s 308us/step - loss: 1.7954
Epoch 10/50
129829/129829 [==============================] - 40s 308us/step - loss: 1.7454
Epoch 11/50
129829/129829 [==============================] -

In [42]:
#model.load_weights('weights-50-0.9411.hdf5')
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001))
print("Learning Rate2= 0.0001 ")
print ("Epoch 50 to 100 \n")

model.fit(X, Y, epochs=100, batch_size=384, initial_epoch=50, callbacks=callbacks_list)

Learning Rate2= 0.0001 
Epoch 50 to 100 

Epoch 51/100
129829/129829 [==============================] - 42s 321us/step - loss: 0.8512
Epoch 52/100
129829/129829 [==============================] - 40s 307us/step - loss: 0.8330
Epoch 53/100
129829/129829 [==============================] - 40s 307us/step - loss: 0.8244
Epoch 54/100
129829/129829 [==============================] - 40s 307us/step - loss: 0.8196
Epoch 55/100
129829/129829 [==============================] - 40s 307us/step - loss: 0.8144
Epoch 56/100
129829/129829 [==============================] - 40s 307us/step - loss: 0.8129
Epoch 57/100
129829/129829 [==============================] - 40s 307us/step - loss: 0.8087
Epoch 58/100
129829/129829 [==============================] - 40s 306us/step - loss: 0.8049
Epoch 59/100
129829/129829 [==============================] - 40s 307us/step - loss: 0.8031
Epoch 60/100
129829/129829 [==============================] - 40s 307us/step - loss: 0.8004
Epoch 61/100
129829/129829 [==========

### Evaluation 
Generating Text with an LSTM Network

In [51]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1) #92908
print("Index of the pattern to start with:",start)
pattern = dataX_padded[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"") 

pattern = list(dataX_padded[start]) #Numpy array to list

# generate characters
import sys
output = ''
for i in range(500):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    output = output+result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

Index of the pattern to start with: 4928
Seed:
" was just in time to hear it s########################################### "


In [52]:
print("Generating Text: \n \n", output)

Generating Text: 
 
 olethmy and bs iave aspu wale tuch lnlely and see as oncered werpenceradle of lt a vhry clarfe of toame of the werling but the waid to the wenl of the senembobtooo as homd the afded a said on her fach on wher stpp anice as speshngs the moew the waid to alice said bester whe hrownan and oore tatsed by tuatted poly as umeer the said tomeiers rres tuleking to the tueenened of toeased herser tenarked out repembir ins tie finis int ttill ie sayher when tuch a pear hirself anice as snee to deer or sle
